In [1]:
#import necessary dependecies
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
import math
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import os
import warnings
import numpy as np  
import seaborn as sns
import pandas as pd, os, gc
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, log_loss
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import StandardScaler, RobustScaler
%matplotlib inline
warnings.filterwarnings('ignore')
from typing import List

In [2]:
#Data Path
data_path ='Data/'
input_path ='/Users/DELL/Desktop/Hackhathons/Zimnat_recommend/'
save_path = 'Data/'
trains='Train.csv'
tests='Test.csv'
submissions='SampleSubmission.csv'

In [3]:
#define some fxn to read the data set
def read_train():
    train =pd.read_csv(data_path + trains, parse_dates=['join_date'])
    return train

def read_test():
    test =pd.read_csv(data_path + tests, parse_dates=['join_date'])
    return test

def read_submission():
    submission =pd.read_csv(data_path + submissions,)
    return submission

#Assign defined fxns to variables
train = read_train();test=read_test();submission=read_submission()

In [4]:
#print all dataset shape
print(train.shape, test.shape,submission.shape)

(29132, 29) (10000, 29) (210000, 2)


In [5]:
#Store Products ID in Multidimensional array
products = train[['P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3']]
products_t = test[['P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3']]

In [6]:
#Axis Transformation
train = train.melt(id_vars=train.columns[:8], value_vars=products, var_name = "PCODE", value_name="Label" )
test = test.melt(id_vars=test.columns[:8], value_vars=products, var_name = "PCODE", value_name="Label" )

In [7]:
#A merger
train['combiner']='x'

In [8]:
#Set
data=pd.concat([train,test],sort=False).reset_index(drop=True)

In [9]:
#preview random 3 sample
data.sample(3)

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,Label,combiner
146744,3XBIOHU,2012-01-09,F,M,1980,E5SW,BPSA,90QI,GYSR,0,x
666120,CSTVP47,2014-01-07,M,M,1982,94KC,0KID,T4MS,GYSR,0,NaN
309116,RGL36O7,2020-01-05,F,W,1969,E5SW,YMGT,T4MS,N2MW,0,x


In [10]:
#Confirm Distribution (TEST AND SAMPLE SUBMISSION FILE)
print('==' * 18);print('TRAIN LABEL DISTRIBUTION');print('==' * 18);print(train['Label'].value_counts())
print('==' * 18);print('TEST LABEL DISTRIBUTION');print('==' * 18);print(test['Label'].value_counts())
print('==' * 18);print('SAMPLE SUBMISSION LABEL DISTRIBUTION');print('==' * 18);print(submission['Label'].value_counts())

TRAIN LABEL DISTRIBUTION
0    545419
1     66353
Name: Label, dtype: int64
TEST LABEL DISTRIBUTION
0    197147
1     12853
Name: Label, dtype: int64
SAMPLE SUBMISSION LABEL DISTRIBUTION
0    197147
1     12853
Name: Label, dtype: int64


In [11]:
train=data[data.combiner.notnull()].reset_index(drop=True);test=data[data.combiner.isna()].reset_index(drop=True)
train.drop('combiner', inplace=True, axis=1);test.drop(['Label','combiner'], inplace=True, axis=1)
print(train.shape, test.shape, submission.shape)

(611772, 10) (210000, 9) (210000, 2)


In [12]:
#Concatenate new column towards submission requirement
train['ID X PCODE'] = train['ID'] + ' X ' + train['PCODE'];test['ID X PCODE'] = test['ID'] + ' X ' + test['PCODE']

In [13]:
#Re-arrange column (not necessary)
train=train[['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'PCODE','ID X PCODE',  'Label']]
test=test[['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'PCODE','ID X PCODE']]

In [14]:
train.head()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE,Label
0,4WKQSBB,2019-01-02,F,M,1987,1X1H,2A7I,T4MS,P5DA,4WKQSBB X P5DA,0
1,CP5S02H,2019-01-06,F,M,1981,UAOD,2A7I,T4MS,P5DA,CP5S02H X P5DA,0
2,2YKDILJ,2013-01-06,M,U,1991,748L,QZYX,90QI,P5DA,2YKDILJ X P5DA,0
3,2S9E81J,2019-01-08,M,M,1990,1X1H,BP09,56SI,P5DA,2S9E81J X P5DA,0
4,BHDYVFT,2019-01-08,M,M,1990,748L,NO3L,T4MS,P5DA,BHDYVFT X P5DA,0


In [15]:
test.head()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE
0,F86J5PC,2018-01-12,M,M,1984,94KC,DZRV,90QI,P5DA,F86J5PC X P5DA
1,H6141K3,2019-01-10,M,M,1996,1X1H,J9SY,90QI,P5DA,H6141K3 X P5DA
2,RBAYUXZ,2020-01-01,F,W,1968,UAOD,2A7I,T4MS,P5DA,RBAYUXZ X P5DA
3,KCBILBQ,2019-01-02,M,M,1989,94KC,2A7I,T4MS,P5DA,KCBILBQ X P5DA
4,LSEC1ZJ,2020-01-02,F,M,1982,UAOD,0KID,T4MS,P5DA,LSEC1ZJ X P5DA


In [16]:
#Lets Check for data co-existence in Train/Test
for col in ['PCODE','occupation_code', 'branch_code','occupation_category_code']:
    train_col = train[col].unique();test_col = test[col].unique()
    print ('Not in Test ' + col + ' ',[i  for i in train_col if i not in test_col])
    print ('Not in Train ' + col + ' ',[i  for i in test_col  if i not in train_col]);print('==' * 30)

Not in Test PCODE  []
Not in Train PCODE  []
Not in Test occupation_code  ['IE90', 'Q0LY', 'JSAX', '6XXU', '8HRZ', 'OQMY', 'INEJ', 'VZN9', 'UC7E', 'PSUY', 'WSRG', 'JQH3', 'LGTN', '738L', 'QQUP', '2XZ1', 'CAAV', 'LLLH', 'W1X2', 'DHSN', 'IX8T', '2US6', 'ZWPL', 'MEFQ', '9B5B', 'JUIP', 'BFD1', 'A4ZC', 'IMHI', 'E5PF', 'GZA8', '3YQ1', 'PJR4', 'NDL9', 'PPNK', '2686', '5LNN', '374O', 'URYD', 'M0WG', 'KBWO', 'ONY7', 'VYSA', 'KUPK', 'R7GL', 'HSVE', 'BER4', '6SKY', 'RH2K', 'ZHC2', 'W3ZV', 'FLXH', 'UYDZ', 'YJXM', '59QM']
Not in Train occupation_code  ['0ZND', '8CHJ', '9F96', 'HSI5', '93OJ', 'BIA0', 'E2MJ', 'JBJP', '0FOI']
Not in Test branch_code  []
Not in Train branch_code  []
Not in Test occupation_category_code  []
Not in Train occupation_category_code  []


In [17]:
train.tail()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE,Label
611767,8SBA93U,2019-01-09,F,M,1974,748L,2A7I,T4MS,ECY3,8SBA93U X ECY3,0
611768,Y73ETTO,2018-01-11,M,M,1987,XX25,Q6J6,90QI,ECY3,Y73ETTO X ECY3,0
611769,MVXG2LC,2019-01-02,M,M,1985,BOAS,6PE7,T4MS,ECY3,MVXG2LC X ECY3,0
611770,BUQ8DQN,2019-01-07,M,M,1989,UAOD,2A7I,T4MS,ECY3,BUQ8DQN X ECY3,0
611771,DEJBWYU,2020-01-03,M,M,1975,94KC,31JW,90QI,ECY3,DEJBWYU X ECY3,0


In [18]:
test.tail()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE
209995,0GMU5UH,2019-01-04,F,M,1986,748L,2A7I,T4MS,ECY3,0GMU5UH X ECY3
209996,I9W11CD,2019-01-08,M,M,1980,UAOD,0KID,T4MS,ECY3,I9W11CD X ECY3
209997,42WTEGT,2018-01-04,M,M,1993,748L,SST3,56SI,ECY3,42WTEGT X ECY3
209998,8EKC4O9,2015-01-05,F,U,1981,49BM,2A7I,T4MS,ECY3,8EKC4O9 X ECY3
209999,WHII93K,2018-01-11,M,U,1977,94KC,2A7I,T4MS,ECY3,WHII93K X ECY3


In [19]:
# LABEL ENCODE
def encode_LE(train,test,cols,verbose=True):
    for col in cols:
        df_comb = pd.concat([train[col],test[col]],axis=0)
        df_comb,_ = df_comb.factorize(sort=True)
        nm = col
        if df_comb.max()>32000: 
            train[nm] = df_comb[:len(train)].astype('int32')
            test[nm] = df_comb[len(train):].astype('int32')
        else:
            train[nm] = df_comb[:len(train)].astype('int16')
            test[nm] = df_comb[len(train):].astype('int16')
        del df_comb; x=gc.collect()
        if verbose: print(nm,', ',end='')

In [20]:
encode_LE(train, test, ['ID','branch_code', 'occupation_code','occupation_category_code','PCODE','sex','marital_status'])

ID , branch_code , occupation_code , occupation_category_code , PCODE , sex , marital_status , 

In [21]:
train.head()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE,Label
0,5384,2019-01-02,0,2,1987,0,19,5,15,4WKQSBB X P5DA,0
1,13834,2019-01-06,0,2,1981,11,19,5,15,CP5S02H X P5DA,0
2,3217,2013-01-06,1,6,1991,3,185,1,15,2YKDILJ X P5DA,0
3,3010,2019-01-08,1,2,1990,0,82,0,15,2S9E81J X P5DA,0
4,12546,2019-01-08,1,2,1990,3,157,5,15,BHDYVFT X P5DA,0


In [22]:
test.head()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE
0,16566,2018-01-12,1,2,1984,4,96,1,15,F86J5PC X P5DA
1,18664,2019-01-10,1,2,1996,0,128,1,15,H6141K3 X P5DA
2,29677,2020-01-01,0,7,1968,11,19,5,15,RBAYUXZ X P5DA
3,22120,2019-01-02,1,2,1989,4,19,5,15,KCBILBQ X P5DA
4,23700,2020-01-02,0,2,1982,11,3,5,15,LSEC1ZJ X P5DA


# TRAIN AND TEST FILE PREPARED

## READ THE INSTRUCTION CAREFULLY AND CRACK THE REST FOR PREDICTION

In [23]:
def insert_year(meth):
    meth.fillna(method='pad', inplace=True)
    meth.insert(loc= 2, column= 'year', value=meth['join_date'].apply(lambda x: x.year))

def insert_month(meth):
    meth.fillna(method='pad', inplace=True)
    meth.insert(loc= 3, column= 'month', value=meth['join_date'].apply(lambda x: x.month))
    
def insert_day(meth):
    meth.fillna(method='pad', inplace=True)
    meth.insert(loc= 2, column= 'day', value=meth['join_date'].apply(lambda x: x.day))

In [24]:
hold_data= [train, test]
for i in hold_data:
    insert_year(i)
    insert_month(i)
    insert_day(i)

In [25]:
test.head()

,ID,join_date,day,year,month,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE
0,16566,2018-01-12,12,2018,1,1,2,1984,4,96,1,15,F86J5PC X P5DA
1,18664,2019-01-10,10,2019,1,1,2,1996,0,128,1,15,H6141K3 X P5DA
2,29677,2020-01-01,1,2020,1,0,7,1968,11,19,5,15,RBAYUXZ X P5DA
3,22120,2019-01-02,2,2019,1,1,2,1989,4,19,5,15,KCBILBQ X P5DA
4,23700,2020-01-02,2,2020,1,0,2,1982,11,3,5,15,LSEC1ZJ X P5DA


In [26]:
train.head()

,ID,join_date,day,year,month,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE,Label
0,5384,2019-01-02,2,2019,1,0,2,1987,0,19,5,15,4WKQSBB X P5DA,0
1,13834,2019-01-06,6,2019,1,0,2,1981,11,19,5,15,CP5S02H X P5DA,0
2,3217,2013-01-06,6,2013,1,1,6,1991,3,185,1,15,2YKDILJ X P5DA,0
3,3010,2019-01-08,8,2019,1,1,2,1990,0,82,0,15,2S9E81J X P5DA,0
4,12546,2019-01-08,8,2019,1,1,2,1990,3,157,5,15,BHDYVFT X P5DA,0


In [27]:
for date in hold_data:
    date.drop(columns=['join_date'], inplace=True)

In [28]:
train.head()

,ID,day,year,month,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE,Label
0,5384,2,2019,1,0,2,1987,0,19,5,15,4WKQSBB X P5DA,0
1,13834,6,2019,1,0,2,1981,11,19,5,15,CP5S02H X P5DA,0
2,3217,6,2013,1,1,6,1991,3,185,1,15,2YKDILJ X P5DA,0
3,3010,8,2019,1,1,2,1990,0,82,0,15,2S9E81J X P5DA,0
4,12546,8,2019,1,1,2,1990,3,157,5,15,BHDYVFT X P5DA,0


In [29]:
from sklearn.model_selection import train_test_split

In [30]:
X_train, X_test, y_train, y_test = train_test_split(train.drop(columns=['ID X PCODE', 'Label']), train['Label'], test_size=0.2, random_state=42)

In [31]:
X_train.head()

,ID,day,year,month,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE
517604,28656,11,2017,1,1,2,1986,7,3,5,11
47532,11453,6,2019,1,0,2,1984,13,164,5,18
369762,30498,2,2017,1,1,2,1985,3,140,1,4
61472,32907,9,2019,1,1,2,1990,1,222,1,2
556184,14310,9,2017,1,1,2,1997,3,19,5,7


# Using Random forest classifier

In [32]:
rd = RandomForestClassifier(n_estimators=20)

In [33]:
rd.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=20,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [34]:
rd.score(X_test, y_test)

0.9579257079808753

In [35]:
test.head()

,ID,day,year,month,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE
0,16566,12,2018,1,1,2,1984,4,96,1,15,F86J5PC X P5DA
1,18664,10,2019,1,1,2,1996,0,128,1,15,H6141K3 X P5DA
2,29677,1,2020,1,0,7,1968,11,19,5,15,RBAYUXZ X P5DA
3,22120,2,2019,1,1,2,1989,4,19,5,15,KCBILBQ X P5DA
4,23700,2,2020,1,0,2,1982,11,3,5,15,LSEC1ZJ X P5DA


In [36]:
data= pd.DataFrame(rd.predict(test.drop(columns=['ID X PCODE'])))

In [37]:
data.insert(loc=0, column='ID X PCODE', value=test['ID X PCODE'])

In [39]:
data.columns= ['ID X PCODE', 'Label']
data.to_csv('Zimnat Insurance Recommendation Challenge.csv', index= False)

In [40]:
data.shape

(210000, 2)

In [41]:
data['Label'].value_counts()

0    190684
1     19316
Name: Label, dtype: int64

In [42]:
submission['Label'].value_counts()

0    197147
1     12853
Name: Label, dtype: int64

In [40]:
from keras import model

ModuleNotFoundError: No module named 'keras'

In [41]:
!pip install keras

In [43]:
from keras import model

ImportError: Keras requires TensorFlow 2.2 or higher. Install TensorFlow via `pip install tensorflow`

In [44]:
!pip install tensorflow

ERROR: Exception:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\pip\_vendor\urllib3\response.py", line 425, in _error_catcher
    yield
  File "C:\ProgramData\Anaconda3\lib\site-packages\pip\_vendor\urllib3\response.py", line 507, in read
    data = self._fp.read(amt) if not fp_closed else b""
  File "C:\ProgramData\Anaconda3\lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 62, in read
    data = self.__fp.read(amt)
  File "C:\ProgramData\Anaconda3\lib\http\client.py", line 457, in read
    n = self.readinto(b)
  File "C:\ProgramData\Anaconda3\lib\http\client.py", line 501, in readinto
    n = self.fp.readinto(b)
  File "C:\ProgramData\Anaconda3\lib\socket.py", line 589, in readinto
    return self._sock.recv_into(b)
  File "C:\ProgramData\Anaconda3\lib\ssl.py", line 1071, in recv_into
    return self.read(nbytes, buffer)
  File "C:\ProgramData\Anaconda3\lib\ssl.py", line 929, in read
    return self._sslobj.read(len, buffer)
sock

In [44]:

probs = model.predict(np.array (x_test)) #predicting x_test in the normal shape 10000 rows

IDXPCODE = list(submissions.iloc[:,0]) #Extracting product names from SubFile

Labels = []

for i in range(test.shape[0]):

    for j in range(21):

        Labels.append(probs[i][j])

        len(Labels) #make sure len is 210000 records

        Submission = pd.DataFrame(list(zip(IDXPCODE,Labels)),columns = ['ID X PCODE', 'Labels'])

        Submission.head(42)

NameError: name 'model' is not defined

In [45]:
from sklearn.ensemble import RandomForestClassfier
from sklearn.feature_selection import SelectFromModel

ImportError: cannot import name 'RandomForestClassfier' from 'sklearn.ensemble' (C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\__init__.py)

In [47]:
from xgboost import XGBClassifier

ModuleNotFoundError: No module named 'xgboost'

In [43]:
model = XGBClassifier(silent=False, 
                      scale_pos_weight=1,
                      learning_rate=0.01,  
                      colsample_bytree = 0.4,
                      subsample = 0.8,
                      objective='binary:logistic', 
                      n_estimators=1000, 
                      reg_alpha = 0.3,
                      max_depth=4, 
                      gamma=10)

NameError: name 'XGBClassifier' is not defined